In [1]:
pip install mpi4py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.4-cp310-cp310-linux_x86_64.whl size=3365665 sha256=63798d05bca0c6b0a3189571d22cc7e051acc922b0d053507a489dab358a7ed3
  Stored in directory: /root/.cache/pip/wheels/e8/1b/b5/97ec4cfccdde26e0f3590ad6e09a5242d508dff09704ef86c1
Successfully built mpi4py


In [2]:
from mpi4py import MPI
from multiprocessing import Pool


In [3]:
def bbp_formula(n):
    """Fórmula de Bailey-Borwein-Plouffe para calcular el n-ésimo dígito hexadecimal de Pi"""
    s = 0.0
    for k in range(n):
        s += (1/(16**k))*((4/(8*k+1)) - (2/(8*k+4)) - (1/(8*k+5)) - (1/(8*k+6)))
    return s


In [4]:

def mpi_pi(digits):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()

    # Calcular la cantidad de dígitos para cada proceso
    digits_per_process = digits // size
    extra_digits = digits % size

    # Calcular el rango de dígitos para el proceso actual
    start_digit = rank * digits_per_process
    end_digit = start_digit + digits_per_process
    if rank == size - 1:
        end_digit += extra_digits

    # Calcular la suma parcial de dígitos en el rango dado
    partial_sum = sum(bbp_formula(digit) for digit in range(start_digit, end_digit))

    # Sumarizar las sumas parciales de cada proceso
    total_sum = comm.reduce(partial_sum, op=MPI.SUM, root=0)

    if rank == 0:
        # Calcular el valor de Pi
        pi = 1 / total_sum

        # Imprimir el resultado
        print(f"Valor de Pi con {digits} dígitos decimales: {pi}")

In [5]:
def multiprocessing_pi(digits):
    pool = Pool()
    result = pool.map(bbp_formula, range(digits))
    pi = 1 / sum(result)
    pool.close()
    pool.join()
    print(f"Valor de Pi con {digits} dígitos decimales: {pi}")

In [6]:
if __name__ == "__main__":
    digits = 1000  # Número de dígitos decimales de Pi a calcular

    print("Resultado utilizando MPI:")
    mpi_pi(digits)

    print("\nResultado utilizando multiprocessing:")
    multiprocessing_pi(digits)

Resultado utilizando MPI:
Valor de Pi con 1000 dígitos decimales: 0.00031862937105431375

Resultado utilizando multiprocessing:
Valor de Pi con 1000 dígitos decimales: 0.00031862937105431375
